In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df=pd.read_csv('CKD.csv')
df=pd.get_dummies(df,dtype=int,drop_first=True)
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [13]:
df.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [3]:
indep=df[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=df['classification_yes']

In [4]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(indep)


In [5]:
from sklearn.preprocessing import StandardScaler
scy= StandardScaler()      
y_train=scy.fit_transform(dep.values.reshape(-1,1))


KNN

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
param_grid = {'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
   }
grid=GridSearchCV(KNeighborsClassifier(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 7, 9], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [7]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(dep,grid.predict_proba(indep)[:,1])
print("The roc_score:",roc_score)

[[150   0]
 [  0 249]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       150
           1       1.00      1.00      1.00       249

    accuracy                           1.00       399
   macro avg       1.00      1.00      1.00       399
weighted avg       1.00      1.00      1.00       399

The f1_macro value for the best parameters {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}: 1.0
The roc_score: 1.0


In [8]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007060,0.002765,0.141244,0.006070,3,1,uniform,"{'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}",0.765903,0.766221,0.790850,0.802640,0.825061,0.790135,0.022520,2
1,0.007928,0.002864,0.124968,0.105968,3,1,distance,"{'n_neighbors': 3, 'p': 1, 'weights': 'distance'}",0.765903,0.766221,0.815438,0.802640,0.812325,0.792505,0.022000,1
2,0.007602,0.001889,0.029931,0.011549,3,2,uniform,"{'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}",0.717004,0.704604,0.814777,0.790000,0.774424,0.760162,0.042487,9
3,0.004695,0.000674,0.009716,0.002309,3,2,distance,"{'n_neighbors': 3, 'p': 2, 'weights': 'distance'}",0.717004,0.692064,0.827369,0.790000,0.800070,0.765302,0.051706,8
4,0.004655,0.001166,0.014267,0.001072,5,1,uniform,"{'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}",0.716619,0.741261,0.778409,0.827652,0.787775,0.770343,0.038432,6
5,0.005408,0.001076,0.008505,0.002542,5,1,distance,"{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}",0.716619,0.765903,0.815187,0.815187,0.812743,0.785128,0.039071,4
6,0.004996,0.001296,0.012674,0.001027,5,2,uniform,"{'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}",0.704545,0.679290,0.765903,0.802640,0.787964,0.748068,0.047984,14
7,0.003391,0.000489,0.007184,0.001498,5,2,distance,"{'n_neighbors': 5, 'p': 2, 'weights': 'distance'}",0.704545,0.679924,0.790829,0.802640,0.775079,0.750604,0.049075,12
8,0.005583,0.002238,0.012769,0.001323,7,1,uniform,"{'n_neighbors': 7, 'p': 1, 'weights': 'uniform'}",0.766221,0.728630,0.717004,0.840046,0.799488,0.770278,0.045409,7
9,0.003990,0.000631,0.006061,0.000670,7,1,distance,"{'n_neighbors': 7, 'p': 1, 'weights': 'distance'}",0.766245,0.753300,0.766221,0.815187,0.825061,0.785203,0.029070,3


In [9]:
age_input=float(input("age:"))
bp_input=float(input("bp:"))
al_input=int(input("al:"))
su_input=int(input("su:"))
bgr_input=float(input("bgr:"))
bu_input=float(input("bu:"))
sc_input=float(input("sc:"))
sod_input=float(input("sod:"))
pot_input=float(input("pot:"))
hrmo_input=float(input("hrmo:"))
pcv_input=int(input("pcv:"))
wc_input=float(input("wc:"))
rc_input=float(input("rc:"))
sg_b_input=int(input("sg_b:"))
sg_c_input=int(input("sg_c:"))
sg_d_input=int(input("sg_d:"))
sg_e_input=int(input("sg_e:"))
rbc_normal_input=float(input("rbc_normal:"))
pc_normal_input=int(input("pc_normal:"))
pcc_present_input=int(input("pcc_present:"))
ba_present_input=int(input("ba_present:"))
htn_yes_input=int(input("htn_yes:"))
dm_yes_input=int(input("dm_yes:"))
cad_yes_input=int(input("cad_yes:"))
appet_yes_input=int(input("appet_yes:"))
pe_yes_input=int(input("pe_yes:"))
ane_yes_input=int(input("ane_yes:"))


In [10]:
import pickle
filename='Grid_ckd_KNN.sav'
f1='classification_report_KNN.sav'
f2='F1_KNN.sav'
f3='roc_KNN.sav'

# Save the objects
pickle.dump(grid, open(filename, 'wb'))
pickle.dump(cr, open(f1, 'wb'))
pickle.dump(fs, open(f2, 'wb'))
pickle.dump(roc_auc_score, open(f3, 'wb'))

# Load the objects
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

with open(f1, 'rb') as file:
    loaded_class_report = pickle.load(file)

with open(f2, 'rb') as file:
    loaded_f1_score = pickle.load(file)

with open(f3, 'rb') as file:
    loaded_roc_auc = pickle.load(file)

In [16]:
future_predictions=grid.predict([[age_input,bp_input,al_input,su_input,bgr_input,bu_input,sc_input,sod_input,pot_input,hrmo_input,pcv_input,wc_input,rc_input,sg_b_input,sg_c_input,sg_d_input,sg_e_input,rbc_normal_input,pc_normal_input,pcc_present_input,ba_present_input,htn_yes_input,dm_yes_input,cad_yes_input,appet_yes_input,pe_yes_input,ane_yes_input]])
print("future_predictions:{}".format(future_predictions))

future_predictions:[1]


C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [17]:
filename="finalized_grid_CKD_KNN.sav"
pickle.dump(grid, open(filename, 'wb'))

In [18]:
loaded_model = pickle.load(open("finalized_grid_CKD_KNN.sav", 'rb'))

In [19]:
preoutput=scy.inverse_transform([future_predictions])
preoutput

array([[1.10842476]])

In [20]:
import pickle
f1='std_scaler_model.pkl'
pickle.dump(sc,open(f1,'wb'))

f2='std_scaler_y_model.pkl'
pickle.dump(scy,open(f2,'wb'))
loaded_model=pickle.load(open('std_scaler_model.pkl','rb'))
loaded_model=pickle.load(open('std_scaler_y_model.pkl','rb'))